<a href="https://colab.research.google.com/github/PimiYvan/Git-assignment/blob/main/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this lab is to implement a language identifier (LID).

Our first model will be based on Naive Bayes.

In [65]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

The next function is used to load the data. Each line of the data consist of a label (corresponding to a language), followed by some text, written in that language. Here is an example of data:

```__label__de Zur Namensdeutung gibt es mehrere Varianten.```


In [2]:
def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    for line in fin:
        tokens = line.split()
        data.append((tokens[0], tokens[1:]))
    return data

You can now try loading the first dataset `train1.txt` and look what examples look like.

In [6]:
data = load_data("train1.txt")
print(data[0])

('__label__de', ['Ich', 'würde', 'alles', 'tun,', 'um', 'dich', 'zu', 'beschützen.'])


Next, we will start implementing the Naive Bayes method. This technique is based on word counts, and we thus need to start by implementing a function to count the words and labels of our training set.

`n_examples` is the total number of examples

`n_words_per_label` is the total number of words for a given label

`label_counts` is the number of times a given label appears in the training data

`word_counts` is the number of times a word appears with a given label

In [53]:
def count_words(data):
    n_examples = 0
    n_words_per_label = defaultdict(lambda: 0)
    label_counts = defaultdict(lambda: 0)
    word_counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for example in data:
        label, sentence = example
        # print(label, sentence)
        
        label_counts[label] += 1
        for word in sentence:
            word_counts[label][word] += 1
        n_words_per_label[label] += len(sentence)
        n_examples += 1
        ## FILL CODE
        # break

    return {'label_counts': label_counts, 
            'word_counts': word_counts, 
            'n_examples': n_examples, 
            'n_words_per_label': n_words_per_label}

Next, using the word and label counts from the previous function, we can implement the prediction function.

Here, `mu` is a regularization parameter (Laplace smoothing), and `sentence` is the list of words corresponding to the test example.

In [66]:
def predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label):
    best_label = None
    best_score = float('-inf')
    i = 0
    res = []
    for label in word_counts.keys():
        score = 0.0
        res = []
        p_j_k = label_counts[label]/n_examples

        for word in sentence:
            score += np.log((word_counts[label][word] + mu)/(label_counts[label] + mu*len(word_counts[label])))
        
        score += np.log(label_counts[label]/n_examples)
        if score >= best_score:
            best_score = score 
            best_label = label
    
    return best_label

The next function will be used to evaluate the Naive Bayes model on a validation set. It computes the accuracy for a particular regularization parameter `mu`.

In [67]:
def compute_accuracy(valid_data, mu, counts):
    accuracy = 0.0
    label_counts = counts['label_counts']
    word_counts = counts['word_counts']
    n_examples = counts['n_examples']
    n_words_per_label = counts['n_words_per_label']
    results = 0
    for label, sentence in valid_data:
        ## FILL CODE
        pred_label = predict(sentence, mu, label_counts, word_counts, n_examples, n_words_per_label)
        if pred_label == label :
            results += 1
        
    return results/len(valid_data)

In [68]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


In [69]:
print("")
print("** Naive Bayes **")
print("")

mu = 1.0
train_data = load_data("train1.txt")
valid_data = load_data("valid1.txt")
counts = count_words(train_data)

print(counts['label_counts'])
print(counts['word_counts'])
print(counts['n_examples'])
print(counts['n_words_per_label'])



** Naive Bayes **

defaultdict(<function count_words.<locals>.<lambda> at 0x7f28f201b7a0>, {'__label__de': 828, '__label__hu': 432, '__label__ru': 1271, '__label__it': 1327, '__label__en': 2137, '__label__es': 564, '__label__tr': 1193, '__label__eo': 1020, '__label__pt': 578, '__label__fr': 650})
defaultdict(<function count_words.<locals>.<lambda> at 0x7f28f201b200>, {'__label__de': defaultdict(<function count_words.<locals>.<lambda>.<locals>.<lambda> at 0x7f28f201bcb0>, {'Ich': 140.0, 'würde': 9.0, 'alles': 6.0, 'tun,': 1.0, 'um': 22.0, 'dich': 15.0, 'zu': 78.0, 'beschützen.': 1.0, 'Tom': 126.0, 'ist': 98.0, 'an': 30.0, 'Kunst': 1.0, 'völlig': 2.0, 'uninteressiert.': 1.0, '„Wird': 1.0, 'das': 62.0, 'in': 57.0, 'der': 93.0, 'Werkstatt': 1.0, 'gemacht?“': 1.0, '–': 8.0, '„Nein,': 1.0, 'muss': 9.0, 'Ort': 3.0, 'und': 68.0, 'Stelle': 1.0, 'erledigt': 1.0, 'werden.“': 1.0, 'Jeder': 2.0, 'liebt': 2.0, 'Musik.': 2.0, 'jetzt': 3.0, 'dreißig.': 1.0, 'Artemis': 1.0, 'Apollos': 1.0, 'Schwester.

In [70]:
print("Validation accuracy: %.3f" % compute_accuracy(valid_data, mu, counts))
print("")

Validation accuracy: 0.939

